<a href="https://colab.research.google.com/github/anruki/AthenAI_Competition/blob/main/Data_Inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparing the Environment

1. Import Libraries

In [2]:
import numpy as np
import random
import os
import zipfile
import pandas as pd

2. Extract zip file

In [4]:
zip_file_path = 'datos_competicion.zip'
extraction_path = 'data'

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print(f"'{zip_file_path}' unzipped to '{extraction_path}'")

'datos_competicion.zip' unzipped to 'data'


## Algoritmo Genético

La función objetivo a maximizar es:

$score=w_1​⋅retornoDiario+w_2​⋅ratioAUM+w3​⋅retornoFuturo$

$retornoDiario$

Mide el crecimiento marginal diario. Como AUM se construye multiplicativamente, un producto con mayor retorno diario domina rápidamente.

* Si w1 es alto → estrategia muy agresiva

* Si w1 es bajo → estrategia más conservadora

$ratioAUM = \frac{ratioAUM_{actual}}{ratioAUM_{inicial}}$

Mide la escala del capital actual. No es retorno, es efecto de tamaño.

Cuando el AUM ya es grande pequeños retornos generan mucho impacto absoluto.

Este término hace que el algoritmo proteja capital grande y evite rotaciones innecesarias.


* w2 alto → más inercia

* w2 bajo → más rotación

$retornoFuturo$ 

Mide el tiempo restante de vida del producto

Un producto que:

está a punto de cerrar tiene poco margen de crecimiento futuro

Favorece productos con vida útil suficiente

* w3 alto → rotación temprana

* w3 bajo → dejar correr productos hasta el final

### Estructura del producto (algoritmo)

In [11]:
class Product:
    def __init__(self, name, open_day, close_day, daily_growth):
        self.name = name
        self.open_day = open_day
        self.close_day = close_day
        self.daily_growth = daily_growth  # ej: 1.001 = +0.1% diario

    def is_active(self, day):
        return self.open_day <= day <= self.close_day

    def days_to_close(self, day):
        return self.close_day - day

## Función Fitness

In [12]:
def simulate_strategy(weights, products, days, initial_capital=100_000):
    capital = initial_capital
    current_product = None

    for day in days:
        active = [p for p in products if p.is_active(day)]
        if not active:
            continue

        scores = []
        for p in active:
            daily_return = p.daily_growth - 1
            ratio_aum = capital / initial_capital
            days_to_close = max(p.days_to_close(day), 1)

            score = (
                weights[0] * daily_return +
                weights[1] * ratio_aum -
                weights[2] * days_to_close
            )
            scores.append(score)

        best_idx = np.argmax(scores)
        best_product = active[best_idx]

        current_product = best_product
        capital *= current_product.daily_growth

    return capital  # AUM final = fitness


In [ ]:
# Inicialización
def init_population(size):
    return [np.random.uniform(-1, 1, 3) for _ in range(size)]

# Selección
def selection(population, fitnesses, elite_ratio=0.2):
    elite_size = int(len(population) * elite_ratio)
    idx = np.argsort(fitnesses)[-elite_size:]
    return [population[i] for i in idx]

# Reporducción
def crossover(parent1, parent2):
    alpha = np.random.rand()
    return alpha * parent1 + (1 - alpha) * parent2

# Mutación
def mutate(individual, rate=0.1):
    if random.random() < rate:
        individual += np.random.normal(0, 0.1, size=len(individual))
    return individual



Loop Evolutivo

In [13]:
def run_ga(products, days,
           population_size=200,
           generations=200):

    population = init_population(population_size)

    for gen in range(generations):
        fitnesses = [
            simulate_strategy(ind, products, days)
            for ind in population
        ]

        best_fitness = max(fitnesses)
        best_ind = population[np.argmax(fitnesses)]

        print(f"Gen {gen:3d} | Best AUM: {best_fitness:,.2f}")

        elites = selection(population, fitnesses)

        new_population = elites.copy()
        while len(new_population) < population_size:
            p1, p2 = random.sample(elites, 2)
            child = crossover(p1, p2)
            child = mutate(child)
            new_population.append(child)

        population = new_population

    return best_ind


Ejecución

In [9]:
products = [
    Product("A", 0, 50, 1.002),
    Product("B", 10, 80, 1.001),
    Product("C", 30, 120, 1.003),
]

days = list(range(0, 150))

best_weights = run_ga(products, days)

print("Pesos óptimos:", best_weights)


Gen   0 | Best AUM: 136,692.18
Gen   1 | Best AUM: 136,692.18
Gen   2 | Best AUM: 136,692.18
Gen   3 | Best AUM: 136,692.18
Gen   4 | Best AUM: 136,692.18
Gen   5 | Best AUM: 136,692.18
Gen   6 | Best AUM: 136,692.18
Gen   7 | Best AUM: 136,692.18
Gen   8 | Best AUM: 136,692.18
Gen   9 | Best AUM: 136,692.18
Gen  10 | Best AUM: 136,692.18
Gen  11 | Best AUM: 136,692.18
Gen  12 | Best AUM: 136,692.18
Gen  13 | Best AUM: 136,692.18
Gen  14 | Best AUM: 136,692.18
Gen  15 | Best AUM: 136,692.18
Gen  16 | Best AUM: 136,692.18
Gen  17 | Best AUM: 136,692.18
Gen  18 | Best AUM: 136,692.18
Gen  19 | Best AUM: 136,692.18
Gen  20 | Best AUM: 136,692.18
Gen  21 | Best AUM: 136,692.18
Gen  22 | Best AUM: 136,692.18
Gen  23 | Best AUM: 136,692.18
Gen  24 | Best AUM: 136,692.18
Gen  25 | Best AUM: 136,692.18
Gen  26 | Best AUM: 136,692.18
Gen  27 | Best AUM: 136,692.18
Gen  28 | Best AUM: 136,692.18
Gen  29 | Best AUM: 136,692.18
Gen  30 | Best AUM: 136,692.18
Gen  31 | Best AUM: 136,692.18
Gen  32 